**Collections**

In [53]:
from collections import defaultdict, Counter, deque, namedtuple
import json

colors = (('Luke', 'Yellow'), ('Paul', 'Blue'), ('James', 'Green'),)

# can pass in a sequence of tuples as well as a dict
fav_colors = defaultdict(str, colors)

fav_colors['Luke'] = 'Red'
# no KeyError is raised
fav_colors['John'] = 'Yellow'
print(json.dumps(fav_colors))

{"Luke": "Red", "Paul": "Blue", "James": "Green", "John": "Yellow"}


In [41]:
tree = lambda: defaultdict(tree)
some_dict = tree()
some_dict['colours']['favourite'] = "yellow"
# json.dumps returns a JSON string representation of the Python object
print(json.dumps(some_dict))

{"colours": {"favourite": "yellow"}}


In [42]:
Counter('abcbab') # a dict sub-class 

Counter({'a': 2, 'b': 3, 'c': 1})

deque provides a queue allowing you to append and delete elements from either side. Values can be popped from either side and the number of items can be limited so that values may be popped rom either side:

In [43]:
d = deque(range(6))
[d.popleft(), d.pop()], d

([0, 5], deque([1, 2, 3, 4]))

In [44]:
d = deque(range(5), maxlen=5)
d.append(5)
d

deque([1, 2, 3, 4, 5])

namedtuple a gives a meaningful name to associate with the object in the tuple instead of only an index value.

In [52]:
color = (55, 155, 255)
Color = namedtuple('Color', ['red', 'green', 'blue'])
named_color = Color(*color)
print (color[0], named_color.red)

55 55


**Copy operations**

Assignment statements in Python do not copy objects, they create bindings between a target and an object. For collections that are mutable or contain mutable items, a copy is sometimes needed so one can change one copy without changing the other.

In [65]:
d = [2, 2, 2]
e = d[:]
d[2] = 4
e

[2, 2, 2]

The difference between shallow and deep copying is only relevant for compound objects (contain other objects, like lists or class instances)

In [66]:
import copy
a = [[2, 2, 2], [2, 2, 2]]
# shallow copy (could use copy.copy). Without the '[:]', 'b' would just point to the object that 'a' references
b = a[:]
c = copy.deepcopy(a)
a[0][2] = 4
b

[[2, 2, 4], [2, 2, 2]]

**Coroutines**

Threads give Python programmers a way to run multiple functions seemingly at the same time but have a number of issues which coroutines can work around. Coroutines let you have many seemingly simultaneous functions in your Python programs. They’re implemented as an extension to generators. Coroutines work by enabling the code consuming a generator to send a value back into the generator function after each yield expression. 

The generator function receives the value passed to the send function as the result of the corresponding yield expression. The initial call to next is required to prepare the generator for receiving the first send by advancing it to the first yield expression. Together, yield and send provide generators with a standard way to vary their next yielded value in response to external input. E.g., to implement a generator coroutine that yields the minimum value it’s been sent so far. Here the bare yield prepares the coroutine with the initial minimum value sent in from the outside. Then the generator repeatedly yields the new minimum in exchange for the next value to consider.

In [7]:
def minimize():
    current = yield
    while True:
        value = yield current
        current = min(value, current)

it = minimize()
next(it)
it.send(20)
it.send(40)
it.send(30)

20

In [8]:
it.send(4)

4

**Decorators**

In [63]:
def a_new_decorator(a_func):
    def wrapTheFunction():
        print("Before a_func()")
        a_func()
        print("After a_func()")
    return wrapTheFunction

def has_name():
    pass

@a_new_decorator
def decorated():
    pass
print(has_name, decorated)

<function has_name at 0x0000024B17599B70> <function a_new_decorator.<locals>.wrapTheFunction at 0x0000024B175AE6A8>


The decorator overrode the name and docstring of the function. This is needed for debugging etc., so to solve this use functools.wraps. The following is a use-case of a decorator, similar to what might be used to protect a route in Flask where if authorization credentials are not contained within the request object a return redirect statement is executed.

In [64]:
from functools import wraps
def decorator_name(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        if not can_run:
            return "Function will not run"
        return f(*args, **kwargs)
    return decorated

@decorator_name
def func():
    return("Function is running")

can_run = True
print(func())

can_run = False
print(func())

Function is running
Function will not run


**Duck-typing** is a concept that states that the type of the object is a matter of concern only at runtime, so that you don’t need to to explicitly mention the type of object performing an operation on it. E.g. an error is only raised for a function being called with unsupported objects if/when it is encountered.

**Factory Methods**
We may not always know what kind of objects we want to create in advance.
Some objects can be created only at execution time after a user requests so. Defer the instantiation of an object until runtime.

**Else clauses on loops**
Think of the else as 'nobreak':

In [ ]:
mylist = [1,2,3]
for i in mylist:
    print(i)
    if i == 3:
        break
else:
    print('Reached StopIteration or broke out of loop')

**functools.partial**

Partials have many uses, but they’re not always obvious. They allow arguments to be pre-filled before they're called in a function. If your function takes x and y, fill in the x beforehand and then call it later with just the y: 

In [6]:
import functools 

def adder(x, y): 
  return x + y 

add_five = functools.partial(adder, y=5)
add_five(3)

8

In [ ]:
**Factory Patterns**

**Function caching**
http://book.pythontips.com/en/latest/function_caching.html

**Iterables and Iterators**
An iterable is an object capable of returning its members one at a time. This includes all sequences types (e.g. str, list etc.), as well as non-sequence types which includes dict, file objects and objects of any class with an __iter__() method.

In [4]:
class MyList(object):
    def __init__(self, lst):
        self.lst = lst
    def __iter__(self):
        return iter(self.lst)

my_iterable = MyList([3,4,5])
[i for i in my_iterable]

[3, 4, 5]

An iterator is an object representing a stream of data.  A classic iterator will be defined using a class with \__iter\__ and \__next\__ methods: 

In [33]:
class Reverser:
    
    def __init__(self, lst):
        self.ind = 0
        self.lst = lst
        
    def __iter__(self):
        return self
    
    def __next__(self):
        self.ind -= 1
        try:
            return self.lst[self.ind]
        except IndexError:
            raise StopIteration
            
r = Reverser([2,3,4])
next(r)

4

Repeated calls to the iterator’s \__next\__() method (or passing it to the built-in function next()) return successive items in the stream. When no more data are available a StopIteration exception is raised. Ways to create an iterative function include: generator expressions, create a generator function (defined using yield instead of return), or by defining a class with \__iter\__ and \__next\__ methods. 

An iterator can be created from a container object (e.g. a list) by passing it to the iter() function. Repeatedly calling the next() method gives a new element each time until a StopIterationor is reached.

In [25]:
x = iter([1, 2, 3])
x.__next__()

1

As iterators have an \__iter\__ method, they may be used in most places where other iterables are accepted and so iterated over in a for loop:

In [27]:
def counter(low, high):
    n = low
    while n <= high:
        yield n
        n += 1
counts = counter(2,8)
[a for a in counts if a % 2 == 0]

[2, 4, 6, 8]

Attempting this with an iterator will just return the same exhausted iterator object used in the previous iteration pass, making it appear empty:

In [29]:
4 in counts

False

**Iterators and Generators**

Generators simplifies creation of iterators. A generator is a function that produces a sequence of results instead of a single value. Each time the yield statement is executed the function generates a new value.


When a generator function is called, it returns a generator object without beginning execution of the function. A generator is also an iterator so you don’t have to worry about the iterator protocol. When next method is called for the first time, the function starts executing until it reaches yield statement. The yielded value is returned by the next() call.

In [36]:
def my_range(n):
    i = 0
    while i < n:
        yield i
        i += 1
m = my_range(10)
# generator object is an iterator so don't need to call iter() on the iterable first
next(m)

0

In [ ]:
**Generator Expressions**
